In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from ex2 import Autoencoder, visualize_reconstruction
import wandb

# 1. Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Using cuda device
cuda


In [2]:
def train(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    wandb.init(
            project="my-awesome-project",
            config={
            "learning_rate": 0.001,
            "architecture": "AutoEncoder",
            "dataset": "FashionMNIST",
            "epochs": epochs,
            }
    )
    wandb.run.log_code(include_fn=lambda path: path.endswith(".py") or path.endswith(".ipynb"))
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, _ in train_loader:  # We don't need the labels for autoencoders
            images = images.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, images)  # Compare the output (reconstructed) with the input

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        val_loss = 0.0
        for images, _ in val_loader:
            images = images.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, images)  # Compare the output (reconstructed) with the input
            val_loss += loss.item()
        wandb.log({"train_loss": running_loss, "val_loss": val_loss})
    wandb.finish()
         
            

In [3]:
# 2. Load Fashion MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [4]:
# Initialize model, loss function, optimizer
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
train(model, train_loader, test_loader, criterion, optimizer, epochs=30)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pawel-magnu (pawel-magnu-agh-university). Use `wandb login --relogin` to force relogin


train_loss,█▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train_loss,554.91599
val_loss,92.6133
